In [1]:
import pyomo.environ as pyo
from idaes.core import FlowsheetBlock
from idaes.models.properties.modular_properties.base.generic_property import (
    GenericParameterBlock)
import idaes.core.util.scaling as iscale
from ammonia_PR import (
    get_prop,
    EosType,
)
from idaes.core.util.model_statistics import (report_statistics,
                                              unfixed_variables_set,
                                              degrees_of_freedom)
from idaes_ui.fv import visualize 

In [ ]:
import importlib
#import Working_DOF1_SOFC.SOFC_unit 
#importlib.reload(Working_DOF1_SOFC.SOFC_unit)
from SOFC_unit_nh3_cracker import SofcUnit


def sofc_example_flowsheet(eos=EosType.PR):
    m = pyo.ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)

    # --- Define component sets for property packages ---
    air_comps = {"O2": 0.21, "N2": 0.79}
    fuel_comps = {"NH3": 0.45, "H2": 0.05, "N2": 0.50}  # Example NH3 fuel composition #"H2O": 0.0
    recycle_comps = {"NH3", "H2", "N2", "H2O"}

    # --- Property packages ---
    m.fs.cathode_side_prop_package = GenericParameterBlock(
        **get_prop(air_comps, {"Vap"}, eos=eos),
        doc="Air property parameters",
    )

    m.fs.anode_side_prop_package = GenericParameterBlock(
        **get_prop(fuel_comps, {"Vap"}, eos=eos),
        doc="Anode side property parameters including NH3",
    )

    m.fs.recycle_prop_package = GenericParameterBlock(
        **get_prop(recycle_comps, {"Vap"}, eos=eos),
        doc="Recycle side property parameters including H2O",
    )

    # --- Instantiate the SOFC unit with NH3 cracker ---
    m.fs.sofc = SofcUnit(
        cathode_side_prop_package=m.fs.cathode_side_prop_package,
        anode_side_prop_package=m.fs.anode_side_prop_package,
        recycle_prop_package=m.fs.recycle_prop_package,
        reaction_eos=eos,
    )
    
    # --- Operating / design parameters ---
    m.fs.sofc.fuel_util.fix(0.77)
    m.fs.sofc.number_of_cells.fix(80)
    m.fs.sofc.cell_area.fix(320)
    m.fs.sofc.dT_Catout_Anout.fix(10)

    # --- Feed ammonia directly into anode_side_inlet ---
    m.fs.sofc.anode_side_inlet.flow_mol.fix(1.0)  # molar flow of ammonia
    m.fs.sofc.anode_side_inlet.temperature.fix(723.15)  # ~450°C cracking temp
    m.fs.sofc.anode_side_inlet.pressure.fix(101325)
    m.fs.sofc.anode_side_inlet.mole_frac_comp[0, "NH3"].fix(0.45)
    m.fs.sofc.anode_side_inlet.mole_frac_comp[0, "N2"].fix(0.50)
    m.fs.sofc.anode_side_inlet.mole_frac_comp[0, "H2"].fix(0.05)

    # --- Air feed to cathode side ---
    m.fs.sofc.cathode_side_inlet.flow_mol.fix(2.0)
    m.fs.sofc.cathode_side_inlet.temperature.fix(948.45)
    m.fs.sofc.cathode_side_inlet.pressure.fix(101325)
    m.fs.sofc.cathode_side_inlet.mole_frac_comp[0, "O2"].fix(0.21)
    m.fs.sofc.cathode_side_inlet.mole_frac_comp[0, "N2"].fix(0.79)
    
    return m



# create a flowsheet with the sofc model
model = sofc_example_flowsheet()


In [3]:
from pyomo.util.infeasible import log_infeasible_constraints
iscale.calculate_scaling_factors(model)
print("\n----- MODEL STATISTICS -----")
print(report_statistics(model))

solver = pyo.SolverFactory("ipopt")
solver.options = {"tol": 1e-6, "max_iter": 1000, "print_level": 5}
results = solver.solve(model, tee=True)
log_infeasible_constraints(model, log_expression=True)



----- MODEL STATISTICS -----

Model Statistics   

Degrees of Freedom: 14 

Total No. Variables: 723 
    No. Fixed Variables: 298
    No. Unused Variables: 129 (Fixed):129)
    No. Variables only in Inequalities: 1 (Fixed: 1) 

Total No. Constraints: 413 
    No. Equality Constraints: 411 (Deactivated: 0)
    No. Inequality Constraints: 2 (Deactivated: 0)

No. Objectives: 0 (Deactivated: 0)

No. Blocks: 109 (Deactivated: 0) 
No. Expressions: 1536 

None
component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.
Ipopt 3.13.2: tol=1e-06
max_iter=1000
print_level=5


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from 

In [ ]:
from pyomo.environ import value

visualize(model.fs.sofc, "Addition of NH3 Cracker to SOFC Unit Model")


print(value(model.fs.sofc.i[:]))              # current density [A m⁻²]
print(value(model.fs.sofc.cell_voltage[:]))   # cell voltage [V]
print(value(model.fs.sofc.sofc_power_dc[:]))  # DC power [W]


2025-07-01 14:14:35 [INFO] idaes.idaes_ui.fv.fsvis: Started visualization server
2025-07-01 14:14:35 [INFO] idaes.idaes_ui.fv.fsvis: Loading saved flowsheet from 'Addition of NH3 Cracker to SOFC Unit Model.json'
2025-07-01 14:14:35 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'Addition of NH3 Cracker to SOFC Unit Model.json' in current directory (c:\Users\Sara\Desktop\IDAES\Nh3Cracker)
2025-07-01 14:14:36 [WARNING] idaes.idaes_ui.fv.flowsheet: Port is neither a feed nor a product: name=o2_poor_strm feed-expr=re.compile('^in_|^feed_|^inlet_|^in$|^feed$|^inlet$|_in$|_feed$|_inlet$', re.IGNORECASE) product-expr=re.compile('^out_|^prod_|^outlet_|^out$|^prod$|^outlet$|_out$|_prod$|_outlet$', re.IGNORECASE)
2025-07-01 14:14:36 [WARNING] idaes.idaes_ui.fv.flowsheet: Port is neither a feed nor a product: name=n2_strm feed-expr=re.compile('^in_|^feed_|^inlet_|^in$|^feed$|^inlet$|_in$|_feed$|_inlet$', re.IGNORECASE) product-expr=re.compile('^out_|^prod_|^outlet_|^out$|^prod$|

2025-07-01 14:14:38 [WARNING] idaes.idaes_ui.fv.flowsheet: Port is neither a feed nor a product: name=o2_poor_strm feed-expr=re.compile('^in_|^feed_|^inlet_|^in$|^feed$|^inlet$|_in$|_feed$|_inlet$', re.IGNORECASE) product-expr=re.compile('^out_|^prod_|^outlet_|^out$|^prod$|^outlet$|_out$|_prod$|_outlet$', re.IGNORECASE)
2025-07-01 14:14:38 [WARNING] idaes.idaes_ui.fv.flowsheet: Port is neither a feed nor a product: name=n2_strm feed-expr=re.compile('^in_|^feed_|^inlet_|^in$|^feed$|^inlet$|_in$|_feed$|_inlet$', re.IGNORECASE) product-expr=re.compile('^out_|^prod_|^outlet_|^out$|^prod$|^outlet$|_out$|_prod$|_outlet$', re.IGNORECASE)
component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.
ERROR: Units problem with expression fs.sofc.cell_voltage[0.0] -
(-434.47*fs.sofc.i[0.0]**5 + 492.56*fs.sofc.i[0.0]**4 -
211.6*fs.sofc.i[0.0]**3 + 44.464*fs.sofc.i[0.0]**2 - 5.4157*fs.sofc.i[0.0] +
1.184375)
component keys 

In [5]:
from idaes.core.util.model_statistics import (
    degrees_of_freedom, 
    report_statistics,
    activated_constraints_set,
    activated_equalities_set,
    unfixed_variables_set,
    fixed_variables_set,
    variables_set,
)


def report_model_dof(model, stream_label, stream):
    separator = '=' * 60
    subsection = '-' * 60

    def format_var(i, var):
        try:
            val = value(var)
            val_str = f"{val: .6e}" if abs(val) < 1e5 else f"{val: .3f}"
        except:
            val_str = "[NaN]"
        return f"{i:>3} - {val_str:>14}   {var}"

    print(f"\n{separator}")
    print(f"{stream_label:^60}")
    print(f"{separator}")

    print("\nSTREAM PROPERTIES:")
    print(f"{'Flow [mol/s]':<20}: {value(stream.flow_mol[0]):.6e}")
    print(f"{'Temperature [K]':<20}: {value(stream.temperature[0]):.2f}")
    print(f"{'Pressure [Pa]':<20}: {value(stream.pressure[0]):.2f}")
    try:
        print(f"{'Heat [W]':<20}: {value(model.control_volume.heat[0]):.2f}")
    except:
        pass

    try:
        print(f"{'Heat of reaction[W]':<20}: {value(model.control_volume.heat_of_reaction[0]):.2f}")
    except:
        pass
    
    
    # print("\nMOLE FRACTIONS:")
    # for t, c in stream.mole_frac_comp.keys():
    #     print(f"  x_{c:<10}: {value(stream.mole_frac_comp[t, c]):.6e}")

    # print(f"\nDOF after {stream_label}: {degrees_of_freedom(model)}")

    # print(f"\n{subsection}")
    # print("MODEL STATISTICS REPORT:")
    # print(subsection)
    # report_statistics(model)

    # print(f"\n{subsection}")
    # print("VARIABLES:")
    # print(subsection)
    # for i, v in enumerate(variables_set(model)):
    #     print(format_var(i + 1, v))

    # print(f"\n{subsection}")
    # print("ACTIVATED CONSTRAINTS:")
    # print(subsection)
    # for i, c in enumerate(activated_constraints_set(model)):
    #     print(f"{i + 1:>3} - {c}")

    # print(f"\n{subsection}")
    # print("ACTIVATED EQUALITIES:")
    # print(subsection)
    # for i, eq in enumerate(activated_equalities_set(model)):
    #     print(f"{i + 1:>3} - {eq}")

    # print(f"\n{subsection}")
    # print("UNFIXED VARIABLES:")
    # print(subsection)
    # for i, ufv in enumerate(unfixed_variables_set(model)):
    #     print(format_var(i + 1, ufv))

    # print(f"\n{subsection}")
    # print("FIXED VARIABLES:")
    # print(subsection)
    # for i, fv in enumerate(fixed_variables_set(model)):
    #     print(format_var(i + 1, fv))

In [6]:
# Esempio di richiamo strutturato per tutte le unità e i flussi associati

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator H2
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.translator_h2, "Translator H2 - Inlet", model.fs.sofc.translator_h2.inlet)
report_model_dof(model.fs.sofc.translator_h2, "Translator H2 - Outlet", model.fs.sofc.translator_h2.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Separator Air (O2 Rich e O2 Poor)
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.separator, "Separator Air - Inlet", model.fs.sofc.separator.inlet)
report_model_dof(model.fs.sofc.separator, "Separator Air - O2 Rich", model.fs.sofc.separator.o2_rich_strm)
report_model_dof(model.fs.sofc.separator, "Separator Air - O2 Poor", model.fs.sofc.separator.o2_poor_strm)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator O2-rich da separator a reaction_props
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.translator_o2, "Translator O2 - Inlet", model.fs.sofc.translator_o2.inlet)
report_model_dof(model.fs.sofc.translator_o2, "Translator O2 - Outlet", model.fs.sofc.translator_o2.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Mixer per H2 e O2 inlet
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.mixer, "Mixer - O2 Inlet", model.fs.sofc.mixer.o2_rich_strm)
report_model_dof(model.fs.sofc.mixer, "Mixer - H2 Inlet", model.fs.sofc.mixer.fuel_strm)
report_model_dof(model.fs.sofc.mixer, "Mixer - Mixed Outlet", model.fs.sofc.mixer.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Stoichiometric Reactor
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.reactor, "Reactor - Inlet", model.fs.sofc.reactor.inlet)
report_model_dof(model.fs.sofc.reactor, "Reactor - Outlet", model.fs.sofc.reactor.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Separator water e O2
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.reactor_separator, "Separator Reactor - O2 Stream", model.fs.sofc.reactor_separator.o2_strm)
report_model_dof(model.fs.sofc.reactor_separator, "Separator Reactor - Water Stream", model.fs.sofc.reactor_separator.water_strm)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator H2O e H2
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.translator_h2_out, "Translator H2 Out - Inlet", model.fs.sofc.translator_h2_out.inlet)
report_model_dof(model.fs.sofc.translator_h2_out, "Translator H2 Out - Outlet", model.fs.sofc.translator_h2_out.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator O2-rich → air
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.translator_o2_out, "Translator O2 Out - Inlet", model.fs.sofc.translator_o2_out.inlet)
report_model_dof(model.fs.sofc.translator_o2_out, "Translator O2 Out - Outlet", model.fs.sofc.translator_o2_out.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Heater condizionamento finale
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.heater, "Heater - Inlet", model.fs.sofc.heater.inlet)
report_model_dof(model.fs.sofc.heater, "Heater - Outlet", model.fs.sofc.heater.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Mixer per O2 riciclato e N2 in uscita
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.mixer_out, "Mixer Outlet - O2 Inlet", model.fs.sofc.mixer_out.o2_strm_final)
report_model_dof(model.fs.sofc.mixer_out, "Mixer Outlet - N2 Inlet", model.fs.sofc.mixer_out.o2_poor_strm)
report_model_dof(model.fs.sofc.mixer_out, "Mixer Outlet - Mixed Outlet", model.fs.sofc.mixer_out.outlet)



                   Translator H2 - Inlet                    

STREAM PROPERTIES:
Flow [mol/s]        : 5.269208e-06
Temperature [K]     : 2261.76
Pressure [Pa]       : 101325.00

                   Translator H2 - Outlet                   

STREAM PROPERTIES:
Flow [mol/s]        : 5.269195e-06
Temperature [K]     : 2261.76
Pressure [Pa]       : 101325.00


AttributeError: '_ScalarSofcUnit' object has no attribute 'separator'